# Part D

## Setup

In [1]:
from py2neo import Graph, ClientError

In [2]:
#set connection variables
PORT = "7687" #database running on this port for bolt connections
USER = "neo4j" #standard user
PASSWORD = "publication-graph" #db password

In [3]:
#connect to database
try:
    graph = Graph('bolt://localhost:'+PORT, auth=(USER, PASSWORD))
    print('SUCCESS: Connected to the Neo4j Database.')
except Exception as e:
    print('ERROR: Could not connect to the Neo4j Database. See console for details.')
    raise SystemExit(e)

SUCCESS: Connected to the Neo4j Database.


In [4]:
# query helper function
def run_query(query:str):
    try:
        return graph.run(query)
    except ClientError as e:
        print(e.message)

## 1. Page Rank

In [5]:
# graph projection
projection_page_rank = """
CALL gds.graph.project(
  'paper-rank',
  'Paper',
  'CITES'
)"""

In [6]:
run_query(projection_page_rank)

Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'paper-rank' already exists.


In [7]:
#page rank computation, parameters: 25 Iterations, dampening factor - 0.85 (standard value)

In [13]:
page_rank_stream = """
CALL gds.pageRank.stream('paper-rank')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).title AS title, score
ORDER BY score DESC, title ASC
"""

In [14]:
cursor = run_query(page_rank_stream)
df = cursor.to_data_frame()

In [16]:
df.head(10)

,title,score
0,Handling shared resources in a temporal data b...,135.675282
1,Can entity-based information systems live with...,128.706558
2,An environment for development of decision sup...,127.707872
3,Classification as a paradigm for computing (ab...,112.368444
4,A reduced dimension branch-and-bound algorithm...,101.967724
5,An inclusive and extensible architecture for e...,101.967724
6,Seven Challenges.,101.967724
7,"The Expressive Power of Urgent, Lazy and Busy-...",101.967724
8,Top-secret Multi-Agent Systems.,101.967724
9,Information structuring and its implementation...,97.527169
